test: 30% 
train-val 70%

To simulate noisy label acquisition, assume a labeler quality p. then use a random 

we first hide the labels of all examples for each dataset. At the point in an experiment when a label is acquired, we generate a label according to the labeler quality p: we assign the example's original label with the probability p and the opposite value with the probability 1−p. 

For each worker, the original true label was assigned to each instance with the probability p, and the opposite value was assigned with the probability 1−𝑝. In our experiments, the labeling quality p of each worker was generated randomly from a uniform distribution on the interval (0.3, 0.9)


After obtaining the labels, we construct the training set to induce a classifier. The classifier is evaluated on the test set (with the true labels). Each experiment is repeated 10 times with a different random data partition, and average results are reported. 

## procedure
1. generate a random p for each labeler
2. generate a random number from a random variable for each instance and if the value is bigger than p, then assign the true label, but if its smaller than p then assign the opposite of true label for that worker 
3.  train a network for these noisy labels for each labeler 
4. repeate this 10 times to be able to measure ethe uncertainty and average accuracy.


## When it is necessary, we convert the target to binary

- for thyroid we keep the negative class and integrate the other three classes into positive; 
- for splice, we integrate classes IE and EI; 
- for waveform, we integrate classes 1 and 2.)

In [1]:
%reload_ext autoreload
%autoreload 2

import sys, os, wget
sys.path.append('../../')

import pandas as pd
import numpy as np
import load_data 
import ipywidgets 
from IPython.display import display

In [2]:
args = {'options':       ['read', 'download'], 
        'label':         'read', 
        'description':   'mode:' ,     
        'orientation':   'vertical',    
        'disabled':      False, 
        'button_style':  '', 
        'layout':        {'width': 'max-content'},
        'tooltips':      ['reading the dataset', 'downloading the dataset'], 
        }

dataset_mode = ipywidgets.ToggleButtons( **args )

display(dataset_mode)

ToggleButtons(description='mode:', layout=Layout(width='max-content'), options=('read', 'download'), tooltips=…

In [30]:
dataset = ipywidgets.Dropdown( options = [ ('1. kr-vs-kp'    ,'kr-vs-kp'), 
                                           ('2. mushroom'    ,'mushroom'),
                                           ('3. sick'        ,'sick'),
                                           ('4. spambase'    ,'spambase'),
                                           ('5. tic-tac-toe' ,'tic-tac-toe'),
                                           ('6. splice'      ,'splice'),
                                           ('7. thyroid'     ,'thyroid'),
                                           ('8. waveform'    ,'waveform'),
                                           ('9. biodeg'      ,'biodeg'),
                                           ('10. horse-colic','horse-colic'),
                                           ('11. ionosphere' ,'ionosphere'),
                                           ('12. vote'       ,'vote')],      
                                value = 'horse-colic')


@ipywidgets.interact(WHICH_DATASET = dataset)
def read_data(WHICH_DATASET):

    if WHICH_DATASET in ['splice','vote','thyroid']:

        print('dataset does not exist')

    else:

        global data, feature_columns
        data, feature_columns = load_data.aim1_3_read_download_UCI_database( WHICH_DATASET = WHICH_DATASET, 
                                                                             mode = dataset_mode.value)
        print(data['train'].head(3) , '\ntrain shape:',data['train'].shape , '\ntest shape:',data['test'].shape)

interactive(children=(Dropdown(description='WHICH_DATASET', index=9, options=(('1. kr-vs-kp', 'kr-vs-kp'), ('2…

In [27]:
data, feature_columns = load_data.aim1_3_read_download_UCI_database(WHICH_DATASET='biodeg', mode='read')


In [31]:
data

,2 1 530101 38.50 66 28 3 3 ? 2 5 4 4 ? ? ? 3 5 45.00 8.40 ? ? 2 2 11300 00000 00000 2
0,1 1 534817 39.2 88 20 ? ? 4 1 3 4 2 ? ? ? 4 2 ...
1,2 1 530334 38.30 40 24 1 1 3 1 3 3 1 ? ? ? 1 1...
2,1 9 5290409 39.10 164 84 4 1 6 2 2 4 4 1 2 5.0...
3,2 1 530255 37.30 104 35 ? ? 6 2 ? ? ? ? ? ? ? ...
4,2 1 528355 ? ? ? 2 1 3 1 2 3 2 2 1 ? 3 3 ? ? ?...
...,...
294,1 1 533886 ? 120 70 4 ? 4 2 2 4 ? ? ? ? ? 5 55...
295,2 1 527702 37.20 72 24 3 2 4 2 4 3 3 3 1 ? 4 4...
296,1 1 529386 37.50 72 30 4 3 4 1 4 4 3 2 1 ? 3 5...
297,1 1 530612 36.50 100 24 3 3 3 1 3 3 3 3 1 ? 4 ...


In [46]:
data, feature_columns = load_data.aim1_3_read_download_UCI_database(WHICH_DATASET='horse-colic', mode='read_raw')

In [47]:
data['train']

,2,1,530101,38.50,66,28,3,3.1,?,2.1,...,45.00,8.40,?.4,?.5,2.2,2.3,11300,00000,00000.1,2.4
1,9,5287179,38.10,136,48,3,3,3,1,5,...,4.90,2,2.90,2,1,2205,0,0,1,NaN
1,1,530478,38.30,44,21,3,1,2,1,3,...,6.50,2,4.40,1,1,2209,0,0,1,NaN
1,1,534885,?,60,36,3,1,3,1,3,...,72,2,1,1,1,2208,0,0,2,NaN
1,9,5290409,39.10,164,84,4,1,6,2,2,...,7.20,3,5.30,2,1,2208,0,0,1,NaN
1,1,530381,?,70,36,1,?,3,2,2,...,7.30,?,?,1,1,31110,0,0,2,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2,9,5305629,38.60,88,28,?,?,?,?,?,...,5.90,?,?,1,2,0,0,0,2,NaN
2,1,533902,38.5,40,16,1,1,1,1,2,...,67,?,?,1,2,0,0,0,2,NaN
2,1,528169,38.20,48,?,2,?,1,2,3,...,6.60,?,?,1,2,3111,0,0,2,NaN
2,1,529628,38.50,60,20,1,1,5,2,2,...,7.50,2,2.30,3,2,300,0,0,1,NaN


In [54]:
pd.read_csv('/groups/jjrodrig/projects/datasets/uci_multilabeler_aim1_3/UCI_horse-colic/horse-colic.data',delimiter=' ', index_col=None)

2  1 530101 38.50 66 28  3 3.1     ? 2.1  ...  \
1 1 534817  39.2  NaN 88 20  ?  ?      4     1  3  4  2   ?     ?   ?  ...   
2 1 530334  38.30 40  24 1   1  3      1     3  3  1  ?   ?     ?   1  ...   
1 9 5290409 39.10 164 84 4   1  6      2     2  4  4  1   2  5.00   3  ...   
2 1 530255  37.30 104 35 ?   ?  6      2     ?  ?  ?  ?   ?     ?   ?  ...   
    528355  ?     ?   ?  2   1  3      1     2  3  2  2   1     ?   3  ...   
...                         .. ..    ...   ... .. .. ..  ..   ...  ..  ...   
1 1 533886  ?     120 70 4   ?  4      2     2  4  ?  ?   ?     ?   ?  ...   
2 1 527702  37.20 72  24 3   2  4      2     4  3  3  3   1     ?   4  ...   
1 1 529386  37.50 72  30 4   3  4      1     4  4  3  2   1     ?   3  ...   
    530612  36.50 100 24 3   3  3      1     3  3  3  3   1     ?   4  ...   
    534618  37.2  40  20 ?   ?  ?      ?     ?  ?  ?  ?   ?     ?   4  ...   

                            45.00 8.40 ?.4  ?.5  2.2     2.3  11300  00000  \
1 1 534817  39.2  NaN 88 20    85    2   2  3.0  2.0  2208.0    0.0    0.0   
2 1 530334  38.30 40  24 1      0    0   1  NaN  NaN     NaN    NaN    NaN   
1 9 5290409 39.10 164 84 4      0    0   1  NaN  NaN     NaN    NaN    NaN   
2 1 530255  37.30 104 35 ?      0    0   2  NaN  NaN     NaN    NaN    NaN   
    528355  ?     ?   ?  2      0    0   2  NaN  NaN     NaN    NaN    NaN   
...                           ...  ...  ..  ...  ...     ...    ...    ...   
1 1 533886  ?     120 70 4      0    0   2  NaN  NaN     NaN    NaN    NaN   
2 1 527702  37.20 72  24 3      0    0   1  NaN  NaN     NaN    NaN    NaN   
1 1 529386  37.50 72  30 4      0    0   2  NaN  NaN     NaN    NaN    NaN   
    530612  36.50 100 24 3      0    0   1  NaN  NaN     NaN    NaN    NaN   
    534618  37.2  40  20 ?      0    0   2  NaN  NaN     NaN    NaN    NaN   

                             00000.1  2.4  
1 1 534817  39.2  NaN 88 20      2.0  NaN  
2 1 530334  38.30 40  24 1       NaN  NaN  
1 9 5290409 39.10 164 84 4       NaN  NaN  
2 1 530255  37.30 104 35 ?       NaN  NaN  
    528355  ?     ?   ?  2       NaN  NaN  
...                              ...  ...  
1 1 533886  ?     120 70 4       NaN  NaN  
2 1 527702  37.20 72  24 3       NaN  NaN  
1 1 529386  37.50 72  30 4       NaN  NaN  
    530612  36.50 100 24 3       NaN  NaN  
    534618  37.2  40  20 ?       NaN  NaN  

[299 rows x 28 columns]

## guide for ipywidgets
source: <https://coderzcolumn.com/tutorials/python/interactive-widgets-in-jupyter-notebook-using-ipywidgets>

In [29]:
widgets.FloatRangeSlider(
    value=[5, 7.5],
    min=0,
    max=10.0,
    step=0.1,
    description='Test:',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='.1f',
)

FloatRangeSlider(value=(5.0, 7.5), continuous_update=False, description='Test:', max=10.0, readout_format='.1f…

In [58]:
b = widgets.BoundedFloatText(
    value=7.5,
    min=0,
    max=10.0,
    step=0.1,
    description='Text:',
    disabled=False
)
b

BoundedFloatText(value=7.5, description='Text:', max=10.0, step=0.1)

In [59]:
b.value 

7.8

In [67]:
a = widgets.ToggleButton(
    value=False,
    description='Run',
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Description',
    icon='check')
a

ToggleButton(value=False, description='Run', icon='check', tooltip='Description')

In [69]:
a.value

True

In [82]:
c = widgets.Checkbox(
    value=False,
    description='this',
    disabled=False,
    indent=False
)

c

Checkbox(value=False, description='this', indent=False)

In [73]:
c.value 

True

In [79]:
widgets.Valid(
    value=True,
    description='Status:',
)

Valid(value=True, description='Status:')

In [84]:
d = widgets.Dropdown(
    options=[('One', 1), ('Two', 2), ('Three', 3)],
    value=2,
    description='Number:')

In [85]:
d.value 

2

In [125]:
widgets.RadioButtons(
    options=['pepperoni', 'pineapple', 'anchovies'],
#    value='pineapple', # Defaults to 'pineapple'
#    layout={'width': 'max-content'}, # If the items' names are long
    description='Pizza topping:',
    disabled=False
)

RadioButtons(description='Pizza topping:', options=('pepperoni', 'pineapple', 'anchovies'), value='pepperoni')

In [128]:
widgets.Box(
    [
        widgets.Label(value='Pizza topping with a very long label:'),
        widgets.RadioButtons(
            options=[
                'pepperoni',
                'pineapple',
                'anchovies',
                'and the long name tha'
            ],
            layout={'width': 'max-content'}
        )
    ]
)

Box(children=(Label(value='Pizza topping with a very long label:'), RadioButtons(layout=Layout(width='max-cont…

In [129]:
widgets.Select(
    options=['Linux', 'Windows', 'OSX'],
    value='OSX',
    # rows=10,
    description='OS:',
    disabled=False
)

Select(description='OS:', index=2, options=('Linux', 'Windows', 'OSX'), value='OSX')

In [135]:
widgets.ToggleButtons(
    options=['Slow', 'Regular', 'Fast'],
    description='Speed:',
    disabled=False,
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
    tooltips=['Description of slow', 'Description of regular', 'Description of fast'],
    icons=['check'] * 3
)

ToggleButtons(description='Speed:', icons=('check', 'check', 'check'), options=('Slow', 'Regular', 'Fast'), to…

In [221]:
a = widgets.ColorPicker(
    concise=False,
    description='Pick a color',
    value='blue',
    disabled=False
)
a

ColorPicker(value='blue', description='Pick a color')

In [223]:
widgets.FileUpload(
    accept='',  # Accepted file extension e.g. '.txt', '.pdf', 'image/*', 'image/*,.pdf'
    multiple=False  # True to accept multiple files upload else False
)

FileUpload(value={}, description='Upload')

In [227]:
from IPython.display import display

def func3(a,b,c):
    display((a+b)^c)

w = interactive(func3,  a=ipywidgets.IntSlider(min=10, max=50, value=25, step=2),
                        b=ipywidgets.IntSlider(min=10, max=50, value=25, step=2),
                        c=ipywidgets.IntSlider(min=10, max=50, value=25, step=2) ) 
display(w)

NameError: name 'interactive' is not defined